In [5]:
import pandas as pd 
import numpy as np
import import_ipynb

from sklearn import tree
from sklearn.impute import SimpleImputer
from functions import *

importing Jupyter notebook from functions.ipynb


In [6]:
train= pd.read_csv("../data/raw/aps_failure_training_set.csv")
test = pd.read_csv("../data/raw/aps_failure_test_set.csv")

print('shape of training set is ',train.shape)
print('shape of test set is ',test.shape)

shape of training set is  (60000, 171)
shape of test set is  (16000, 171)


## Data Preprocessing

1. Replace 'na' with null value and convert column into numeric data type
2. Imputing missing variables with mean

In [7]:
###defining target variables(y) and predictor variables (x)
y_train = train['class']
X_train = train.drop('class',axis=1)

y_test = test['class']
X_test = test.drop('class',axis=1)

In [8]:
X_train.head()

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,76698,na,2130706438,280,0,0,0,0,0,0,...,1240520,493384,721044,469792,339156,157956,73224,0,0,0
1,33058,na,0,na,0,0,0,0,0,0,...,421400,178064,293306,245416,133654,81140,97576,1500,0,0
2,41040,na,228,100,0,0,0,0,0,0,...,277378,159812,423992,409564,320746,158022,95128,514,0,0
3,12,0,70,66,0,10,0,0,0,318,...,240,46,58,44,10,0,0,0,4,32
4,60874,na,1368,458,0,0,0,0,0,0,...,622012,229790,405298,347188,286954,311560,433954,1218,0,0


In [16]:
## Replace 'na' with null value and convert column into numeric data type

X_train.replace('na',np.NaN,inplace=True)
X_train=X_train[X_train.columns].apply(pd.to_numeric, errors='coerce')

X_test.replace('na',np.NaN,inplace=True)
X_test=X_test[X_train.columns].apply(pd.to_numeric, errors='coerce')

In [17]:
## imputing mising variables
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

X_train_new_adjusted= pd.DataFrame(imp.fit_transform(X_train),columns=X_train.columns)
X_test_new_adjusted= pd.DataFrame(imp.fit_transform(X_test),columns=X_test.columns)

# The super fancy but copy-pastable machine learning

In [18]:
## Training/Fitting model using training data

clf = tree.DecisionTreeClassifier()
clf.fit(X_train_new_adjusted,y_train)

DecisionTreeClassifier()

In [20]:
## Using model to generate prediction using test data predictor variables

y_pred = clf.predict(X_test)
y_pred=pd.DataFrame(y_pred)

In [13]:
##Compute test dataset cost

cost(y_test,y_pred)

wrongly labelled positives = 70
wrongly labelled negatives = 109
total cost of wrongly labelling = 55200


# JW's section

## SMOTE

In [26]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
smote = SMOTE()

In [27]:
def balance_data(df,label):
    over = SMOTE(sampling_strategy=0.3)
    under = RandomUnderSampler(sampling_strategy=0.5)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    df, label = pipeline.fit_resample(df, label)
    return df, label

In [28]:
X_train_smote, y_train_smote = balance_data(X_train_new_adjusted,y_train)

In [30]:
## Testing
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_smote,y_train_smote)
y_pred = clf.predict(X_test)
y_pred = pd.DataFrame(y_pred)
cost(y_test,y_pred)

wrongly labelled positives = 191
wrongly labelled negatives = 82
total cost of wrongly labelling = 42910
